In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Airmean_chennai.csv")

In [3]:
dataset

,Date,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
0,2015-01-01,Chennai,77.394417,149.388739,16.30,15.39,22.68,4.59,1.17,9.20,11.35000,0.170000,7.402406,1.197632,170.088957
1,2015-01-02,Chennai,77.394417,149.388739,16.49,13.42,23.09,7.83,1.23,8.61,9.16000,0.130000,7.402406,1.197632,170.088957
2,2015-01-03,Chennai,77.394417,149.388739,9.72,19.56,9.99,4.63,0.77,20.56,13.45000,0.030000,7.402406,1.197632,170.088957
3,2015-01-04,Chennai,77.394417,149.388739,9.60,16.20,11.71,5.23,1.00,20.56,10.33000,2.120333,7.402406,1.197632,170.088957
4,2015-01-05,Chennai,77.394417,149.388739,9.16,16.30,12.94,5.50,0.90,16.60,9.36000,2.120333,7.402406,1.197632,170.088957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,2020-06-27,Chennai,26.420000,83.998141,7.25,12.96,19.59,33.20,1.10,7.29,68.51000,0.100000,0.070000,1.197632,95.000000
2005,2020-06-28,Chennai,25.930000,83.998141,7.81,10.00,16.39,35.98,0.76,6.48,72.40625,0.090000,0.000000,1.197632,98.000000
2006,2020-06-29,Chennai,21.300000,83.998141,7.65,9.69,16.74,34.07,0.96,6.62,62.57000,0.090000,0.010000,1.197632,104.000000
2007,2020-06-30,Chennai,24.140000,83.998141,8.42,12.38,20.29,34.17,1.05,7.50,68.75000,0.170000,0.160000,1.197632,110.000000


In [4]:
dataset["City"].value_counts()

City
Chennai    2009
Name: count, dtype: int64

In [5]:
dataset.isnull().sum()

Date       0
City       0
PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
CO         0
SO2        0
O3         0
Benzene    0
Toluene    0
Xylene     0
AQI        0
dtype: int64

In [6]:
timedata=pd.DataFrame()

In [7]:
timedata["Date"]=dataset["Date"]
timedata["AQI"]=dataset["AQI"]

In [8]:
timedata

,Date,AQI
0,2015-01-01,170.088957
1,2015-01-02,170.088957
2,2015-01-03,170.088957
3,2015-01-04,170.088957
4,2015-01-05,170.088957
...,...,...
2004,2020-06-27,95.000000
2005,2020-06-28,98.000000
2006,2020-06-29,104.000000
2007,2020-06-30,110.000000


In [9]:
data=timedata["AQI"]

In [10]:
data_train=data.head(1500)

In [11]:
data_train

0       170.088957
1       170.088957
2       170.088957
3       170.088957
4       170.088957
           ...    
1495     86.000000
1496    115.000000
1497    149.000000
1498    138.000000
1499     87.000000
Name: AQI, Length: 1500, dtype: float64

In [12]:
data_test=data.tail(509)

In [13]:
data_test

1500     77.0
1501     67.0
1502     77.0
1503    126.0
1504    200.0
        ...  
2004     95.0
2005     98.0
2006    104.0
2007    110.0
2008     92.0
Name: AQI, Length: 509, dtype: float64

In [14]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(data_train, order=(1,1,1), seasonal_order=(0,0,0,0))#Seasonal Autoregressive Integrated Moving Average (ARIMA)
model_fit = model.fit()
# make prediction
y_pred= model_fit.predict(len(data_train), len(data)-1)
print(y_pred)
from sklearn.metrics import r2_score
r2=r2_score(data_test,y_pred)
print("r2={}".format(r2))

1500    101.526720
1501    110.106999
1502    115.174982
1503    118.168411
1504    119.936495
           ...    
2004    122.487709
2005    122.487709
2006    122.487709
2007    122.487709
2008    122.487709
Name: predicted_mean, Length: 509, dtype: float64
r2=-0.6459179344069113


In [17]:
trends=['n','t','c','ct']
orders=[(0,0,0),(0,0,1),(2,0,1),(1,1,1)]#non-seasonal ARIMA order parameters:
orderslist=[]
trendslist=[]
rscorelist=[]
for td in trends:
    for i in orders:
        trendslist.append(td)
        orderslist.append(i)
        from statsmodels.tsa.statespace.sarimax import SARIMAX
        model = SARIMAX(data_train, order=i, seasonal_order=(0, 0, 0, 0), trend=td)#(0, 0, 0, 0) means that the SARIMAX model does not have any seasonal components.
        model_fit = model.fit()
    # make prediction
        y_pred= model_fit.predict(len(data_train), len(data)-1)
        print(y_pred)
        from sklearn.metrics import r2_score
        r2=r2_score(data_test,y_pred)
        rscorelist.append(r2)
        print("Trend={}, Order={} ,r2={}".format(td,i,r2))

1500    0.0
1501    0.0
1502    0.0
1503    0.0
1504    0.0
       ... 
2004    0.0
2005    0.0
2006    0.0
2007    0.0
2008    0.0
Name: predicted_mean, Length: 509, dtype: float64
Trend=n, Order=(0, 0, 0) ,r2=-6.410263024207206
1500    22.87763
1501     0.00000
1502     0.00000
1503     0.00000
1504     0.00000
          ...   
2004     0.00000
2005     0.00000
2006     0.00000
2007     0.00000
2008     0.00000
Name: predicted_mean, Length: 509, dtype: float64
Trend=n, Order=(0, 0, 1) ,r2=-6.405887104312267
1500    101.286217
1501    109.713797
1502    114.682691
1503    117.609728
1504    119.331342
           ...    
2004    114.287975
2005    114.273360
2006    114.258746
2007    114.244134
2008    114.229525
Name: predicted_mean, Length: 509, dtype: float64
Trend=n, Order=(2, 0, 1) ,r2=-0.4405538072761961
1500    101.526720
1501    110.106999
1502    115.174982
1503    118.168411
1504    119.936495
           ...    
2004    122.487709
2005    122.487709
2006    122.487709
2007  

C:\Users\Lenovo\anaconda3\envs\aiml\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


1500     99.906276
1501    106.907186
1502    110.784144
1503    113.005344
1504    114.345910
           ...    
2004    124.668784
2005    124.668784
2006    124.668784
2007    124.668784
2008    124.668784
Name: predicted_mean, Length: 509, dtype: float64
Trend=c, Order=(2, 0, 1) ,r2=-0.738612108859946
1500    100.929146
1501    109.154506
1502    114.005788
1503    116.861134
1504    118.535783
           ...    
2004    103.346323
2005    103.310991
2006    103.275659
2007    103.240327
2008    103.204995
Name: predicted_mean, Length: 509, dtype: float64
Trend=c, Order=(1, 1, 1) ,r2=-0.22730206816005483
1500    91.569963
1501    91.523941
1502    91.477918
1503    91.431895
1504    91.385872
          ...    
2004    68.374403
2005    68.328380
2006    68.282357
2007    68.236334
2008    68.190312
Name: predicted_mean, Length: 509, dtype: float64
Trend=ct, Order=(0, 0, 0) ,r2=-0.07528909495250002
1500    83.705089
1501    92.530728
1502    92.486731
1503    92.442735
1504    92.39

In [18]:
rscorelist

[-6.410263024207206,
 -6.405887104312267,
 -0.4405538072761961,
 -0.6459179344069113,
 -9.10667903234865,
 -9.07944079728101,
 -1.3865960613544073,
 -0.3247818363652779,
 -0.8177724908589707,
 -0.8152342645817721,
 -0.738612108859946,
 -0.22730206816005483,
 -0.07528909495250002,
 -0.04811386590587374,
 -0.08580689591517654,
 -4.136521895071486]

In [19]:
result=pd.DataFrame()

In [20]:
result.insert(0,"Trend",trendslist)
result.insert(1,"Order",orderslist)
result.insert(2,"R_score",rscorelist)

In [21]:
result

,Trend,Order,R_score
0,n,"(0, 0, 0)",-6.410263
1,n,"(0, 0, 1)",-6.405887
2,n,"(2, 0, 1)",-0.440554
3,n,"(1, 1, 1)",-0.645918
4,t,"(0, 0, 0)",-9.106679
5,t,"(0, 0, 1)",-9.079441
6,t,"(2, 0, 1)",-1.386596
7,t,"(1, 1, 1)",-0.324782
8,c,"(0, 0, 0)",-0.817772
9,c,"(0, 0, 1)",-0.815234


In [22]:
model_fit.predict(len(data), len(data)+5)

2009    205.639092
2010    205.850491
2011    206.062108
2012    206.273945
2013    206.486000
2014    206.698274
Name: predicted_mean, dtype: float64